# Group 09 Project Notebook
# Members: Chan Keng Jit, Vikas Harlani, Toh Yi Zhi, Yap Zi Chen, Zhang Xiao

## Import Statements

Import relevant libraries for the project

In [ ]:
# CatBoost install
!pip install catboost
!pip install xgboost
!pip install ipywidgets

In [2]:
# Import Statements
import numpy as np
import scipy as sp
import pandas as pd
from pandas.plotting import scatter_matrix
pd.options.mode.chained_assignment = None

import sklearn
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report,roc_auc_score,confusion_matrix,accuracy_score,f1_score,roc_curve
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import itertools

from scipy import stats
import scipy.stats as sct
from scipy.stats import norm, expon, uniform
from scipy.optimize import minimize
import re
from sklearn import svm
import seaborn as sb

# Imports for preprocessing
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
# Imports for svr
from sklearn import svm
# Imports for linear regression
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
# Imports for random forest
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

# Imports for metrics
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import KFold, cross_val_score

pd.options.mode.chained_assignment = None
np.random.seed(10)

# Imports for catboostregressor
from catboost import CatBoostRegressor
import ipywidgets as ipw # not sure if this is used

# Set random seed for reproducability
np.random.seed(42)

## Load train and test datasets from CSV files

In [5]:
Data_train = pd.read_csv('Data-train.csv', index_col=0)
Data_train.head()

,productID,brandID,attribute1,attribute2,attribute3,attribute4,clickVolume,avgOriginalUnitPrice,avgFinalUnitPrice,ma14SalesVolume,weekday,meanAge,gender,meanEducation,maritalStatus,plus,meanPurchasePower,meanUserLevel,meanCityLevel,sales
0,68,28,3.0,100.0,9,1.561599,799.0,237.206500,186.956968,11.785714,4,35.498327,0.114401,3.046749,0.562923,0.206291,2.173474,3.628150,2.181482,14
1,31,6,3.0,100.0,2,7.747685,352.0,178.540538,129.123143,25.785714,7,25.633478,0.000000,3.005766,0.000000,0.444095,1.927769,3.058834,1.511026,2
2,27,19,3.0,60.0,2,3.899665,293.0,119.066221,59.136199,21.357143,6,32.228120,0.158911,2.935705,0.293093,0.132959,2.324772,2.650857,1.665037,16
3,52,4,3.0,80.0,1,1.585429,537.0,139.452545,68.572100,7.285714,5,29.304317,0.186522,2.810752,0.242181,0.000000,2.192938,2.026259,2.481082,13
4,3,28,3.0,100.0,14,4.623051,6668.0,298.106532,244.047761,296.214286,5,29.132081,0.300183,3.032527,0.391279,0.103062,2.135294,2.736957,1.979200,159


In [6]:
Data_test = pd.read_csv('Data-test.csv', index_col=0)
Data_test.head()

,productID,brandID,attribute1,attribute2,attribute3,attribute4,clickVolume,avgOriginalUnitPrice,avgFinalUnitPrice,ma14SalesVolume,weekday,meanAge,gender,meanEducation,maritalStatus,plus,meanPurchasePower,meanUserLevel,meanCityLevel
0,77,38,3.0,100.0,2,5.670374,173.0,180.835683,165.459988,12.857143,4,26.960521,0.053158,2.958528,0.668194,0.148227,2.347953,2.638135,1.997034
1,47,30,4.0,100.0,9,5.191429,142.0,190.802856,138.995392,2.285714,5,33.111475,0.317086,3.292282,0.709143,0.272464,2.409795,3.973464,1.385002
2,33,25,3.0,100.0,1,2.461109,144.0,127.385574,123.286542,4.500000,6,30.353550,0.000000,2.668048,1.002511,0.000000,1.955541,2.937542,2.369596
3,68,28,3.0,100.0,2,4.659192,220.0,240.873276,168.258175,17.500000,6,29.692379,0.000000,2.470400,0.424764,0.000000,1.932938,2.480643,2.505191
4,65,16,3.0,90.0,2,15.554293,4079.0,85.013188,68.521885,115.785714,7,33.128355,0.201974,2.432218,0.592206,0.005535,2.428117,1.792081,2.669685


## Data Exploration


Data Exploration, and Feature Visualization is done to better understand the features and to come up with ideas for effective feature representation. Data cleaning is also done.

### Dropping of rows with missing data

In [ ]:
print('shape of Data_train: ', Data_train.shape)
print('shape of Data_test:', Data_test.shape)

In [6]:
# Drop any rows that have missing data in any columns

Data_train.dropna()
Data_test.dropna()

,productID,brandID,attribute1,attribute2,attribute3,attribute4,clickVolume,avgOriginalUnitPrice,avgFinalUnitPrice,ma14SalesVolume,weekday,meanAge,gender,meanEducation,maritalStatus,plus,meanPurchasePower,meanUserLevel,meanCityLevel
0,77,38,3.0,100.0,2,5.670374,173.0,180.835683,165.459988,12.857143,4,26.960521,0.053158,2.958528,0.668194,0.148227,2.347953,2.638135,1.997034
1,47,30,4.0,100.0,9,5.191429,142.0,190.802856,138.995392,2.285714,5,33.111475,0.317086,3.292282,0.709143,0.272464,2.409795,3.973464,1.385002
2,33,25,3.0,100.0,1,2.461109,144.0,127.385574,123.286542,4.500000,6,30.353550,0.000000,2.668048,1.002511,0.000000,1.955541,2.937542,2.369596
3,68,28,3.0,100.0,2,4.659192,220.0,240.873276,168.258175,17.500000,6,29.692379,0.000000,2.470400,0.424764,0.000000,1.932938,2.480643,2.505191
4,65,16,3.0,90.0,2,15.554293,4079.0,85.013188,68.521885,115.785714,7,33.128355,0.201974,2.432218,0.592206,0.005535,2.428117,1.792081,2.669685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,17,29,2.0,50.0,1,7.783232,212.0,70.289424,58.684552,5.357143,6,26.794008,0.558049,2.629016,0.193424,0.332677,2.640429,1.722599,1.758180
1047,66,20,4.0,100.0,6,17.327509,1044.0,210.529395,174.739314,10.071429,5,34.052881,0.164406,3.094639,0.510198,0.274645,2.209462,2.788644,2.101402
1048,68,28,3.0,100.0,2,1.200506,1040.0,235.563711,190.538695,3.571429,5,38.025742,0.371263,3.073274,0.813218,0.422813,2.088130,3.622261,1.577160
1049,68,28,3.0,100.0,2,1.076817,324.0,239.301695,192.178675,6.428571,2,35.851558,0.519561,2.967437,0.739173,0.320380,1.982423,2.999338,1.278792


In [ ]:
print('shape of Data_train: ', Data_train.shape)
print('shape of Data_test:', Data_test.shape)

Since the shape is the same before and after the .dropna() command, there are no rows with missing data.

### Ordinary Least Squares Regression

We did an ordinary least squares regression model initially on the raw train dataset in order to explore the dataset.

In [8]:
def get_smf_str(y_str, x_list, x_list_remove = []):
    x_str = ''
    for i in range(len(x_list)):
        if x_list[i] in x_list_remove:
            continue
        x_str += x_list[i]
        if i != len(x_list) - 1:
            x_str += ' + '
    smf_str = ''
    smf_str += y_str
    smf_str += ' ~ '
    smf_str += x_str
    return smf_str
y_str = Data_train.columns[-1]
x_list = Data_train.columns[:-1]

In [ ]:
results = smf.ols(get_smf_str(y_str, x_list), data=Data_train).fit()
print(results.summary())

In [ ]:
results.pvalues

In [ ]:
Data_train.dtypes

The P-values of all the predictors are obtained as follows, with clickVolume observed to have an extremely low P-value as compared to the other predictor variables.

### Exploring the number of unique values in each feature

In order to determine which features are likely categorical, we explored the number of unique values in each feature. The features which only have a few unique values are thought to be categorical.

In [ ]:
Data_train.nunique()

In [ ]:
Data_test.nunique()

Findings:

attribute1 has only 4 unique values in both the training and test dataset -> likely to be categorical.

attribute2 has only 7 unique values in both the training and test dataset -> likely to be categorical.

attribute 3 has 26 unique values in the training dataset and 24 unique values in the test dataset -> likely to be categorical.

weekday has only 7 unique values (and specifies the day of the week) -> categorical.

productID and brandID are categorical features as well.

### Correlation Matrix

A correlation heatmap is plotted for the training dataset, using all the features. This is to determine which features have higher correlation.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize = (16, 8))
df = pd.DataFrame(Data_train)
corr = df.corr()
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap Matrix', fontdict={'fontsize':12}, pad=14);

The correlation heatmap matrix shows that ‘avgFinalUnitPrice’ has a strong correlation with ‘avgOriginalUnitPrice’ at 0.93. Furthermore, there is a high correlation between ‘clickVolume’ and ‘sales’ at 0.90, as well as ‘attribute1’ and ‘attribute2’, at 0.55.

For such attributes, they would be retained in order for us to get a general comprehensive overview of the model performance with all the features included.


### Scatter Plot

A scatter plot of the training dataset is plotted to visualize the different features.

In [ ]:
scatter_matrix(Data_train, figsize = (20,20))

## Data Pre-Processing

Using insights from the data exploration, we performed data pre-processing before training and evaluating different models. We obtained two seperate processed datasets, one in which we perform one-hot encoding to the (supposed) categorical features and one in which we don't. This is to allow us to evaluate the performance of the models with both approaches.

Let the one-hot encoding dataset be called OHE and the non encoded dataset be called Stan.

In [7]:
# Extract X_train, y_train and X_test dataframes from training and test dataset

X_train = Data_train.drop('sales', axis = 1)
y_train = Data_train.sales

X_test = Data_test

### Stan Dataset

In the Stan processed dataset, we standardise the features so that the standard deviation of each feature is now one. However, we do not perform encoding for the categorical variables. The StandardScaler library is used and is fit onto only X_train, but used to transform both X_train and X_test.

In [ ]:
ss=StandardScaler(with_mean=False)

ss.fit(X_train)

Stan_X_train = ss.transform(X_train)
Stan_y_train = Data_train.sales
Stan_X_test = ss.transform(X_test)

Stan_X_train_df = pd.DataFrame(Stan_X_train, columns=X_train.columns)  # Assuming X_train is a DataFrame
Stan_y_train_df = pd.DataFrame(Stan_y_train, columns=['sales'])
Stan_X_test_df = pd.DataFrame(Stan_X_test, columns=X_test.columns)  # Assuming X_train is a DataFrame

print(Stan_X_train_df.head())
print(Stan_X_test_df.head())


### One-Hot Encoding Dataset

In the One-Hot Encoding Dataset, we perform One-Hot Encoding for the categorical features and then perform standardization.

The (assumed) categorical predictors are:

1. weekday
2. product_ID
3. brand_ID
4. attribute1
5. atrribute2
6. attribute3


#### One-Hot Encoding

The train dataset and test dataset was first combined, as the One-Hot Encoding could be performed together.


In [9]:
Combined = pd.concat([X_train,X_test])

Conversion of categorical predictors to a String type to be utilised as One-Hot Encoding (OHE)

In [10]:
Combined.weekday = Combined.weekday.astype(str)
Combined.productID = Combined.productID.astype(str)
Combined.brandID = Combined.brandID.astype(str)
Combined.attribute1 = Combined.attribute1.astype(str)
Combined.attribute2 = Combined.attribute2.astype(str)
Combined.attribute3 = Combined.attribute3.astype(str)
Combined=pd.get_dummies(Combined)
Combined.head()

,attribute4,clickVolume,avgOriginalUnitPrice,avgFinalUnitPrice,ma14SalesVolume,meanAge,gender,meanEducation,maritalStatus,plus,...,attribute3_7,attribute3_8,attribute3_9,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1.561599,799.0,237.206500,186.956968,11.785714,35.498327,0.114401,3.046749,0.562923,0.206291,...,False,False,True,False,False,False,True,False,False,False
1,7.747685,352.0,178.540538,129.123143,25.785714,25.633478,0.000000,3.005766,0.000000,0.444095,...,False,False,False,False,False,False,False,False,False,True
2,3.899665,293.0,119.066221,59.136199,21.357143,32.228120,0.158911,2.935705,0.293093,0.132959,...,False,False,False,False,False,False,False,False,True,False
3,1.585429,537.0,139.452545,68.572100,7.285714,29.304317,0.186522,2.810752,0.242181,0.000000,...,False,False,False,False,False,False,False,True,False,False
4,4.623051,6668.0,298.106532,244.047761,296.214286,29.132081,0.300183,3.032527,0.391279,0.103062,...,False,False,False,False,False,False,False,True,False,False


Split back to training and testing datasets

In [11]:
transformed_X_train = Combined.iloc[:Data_train.shape[0]]
transformed_X_train

,attribute4,clickVolume,avgOriginalUnitPrice,avgFinalUnitPrice,ma14SalesVolume,meanAge,gender,meanEducation,maritalStatus,plus,...,attribute3_7,attribute3_8,attribute3_9,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1.561599,799.0,237.206500,186.956968,11.785714,35.498327,0.114401,3.046749,0.562923,0.206291,...,False,False,True,False,False,False,True,False,False,False
1,7.747685,352.0,178.540538,129.123143,25.785714,25.633478,0.000000,3.005766,0.000000,0.444095,...,False,False,False,False,False,False,False,False,False,True
2,3.899665,293.0,119.066221,59.136199,21.357143,32.228120,0.158911,2.935705,0.293093,0.132959,...,False,False,False,False,False,False,False,False,True,False
3,1.585429,537.0,139.452545,68.572100,7.285714,29.304317,0.186522,2.810752,0.242181,0.000000,...,False,False,False,False,False,False,False,True,False,False
4,4.623051,6668.0,298.106532,244.047761,296.214286,29.132081,0.300183,3.032527,0.391279,0.103062,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571,1.610909,1558.0,89.371417,41.336335,34.142857,38.176047,0.434893,2.805286,0.652032,0.678596,...,False,False,False,False,False,False,False,False,True,False
1572,13.657958,74.0,80.494579,59.261375,5.214286,20.134348,0.019198,2.969021,0.052638,0.065188,...,False,False,False,False,False,False,True,False,False,False
1573,5.449706,353.0,76.912038,48.459064,5.000000,35.180962,0.260036,2.425446,0.701358,0.190540,...,False,False,False,True,False,False,False,False,False,False
1574,9.496754,1740.0,69.920092,50.999694,59.857143,31.277144,0.639961,3.125656,0.678429,0.319564,...,True,False,False,False,False,False,False,False,True,False


In [12]:
transformed_X_test = Combined.iloc[Data_train.shape[0]:]
transformed_X_test

,attribute4,clickVolume,avgOriginalUnitPrice,avgFinalUnitPrice,ma14SalesVolume,meanAge,gender,meanEducation,maritalStatus,plus,...,attribute3_7,attribute3_8,attribute3_9,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,5.670374,173.0,180.835683,165.459988,12.857143,26.960521,0.053158,2.958528,0.668194,0.148227,...,False,False,False,False,False,False,True,False,False,False
1,5.191429,142.0,190.802856,138.995392,2.285714,33.111475,0.317086,3.292282,0.709143,0.272464,...,False,False,True,False,False,False,False,True,False,False
2,2.461109,144.0,127.385574,123.286542,4.500000,30.353550,0.000000,2.668048,1.002511,0.000000,...,False,False,False,False,False,False,False,False,True,False
3,4.659192,220.0,240.873276,168.258175,17.500000,29.692379,0.000000,2.470400,0.424764,0.000000,...,False,False,False,False,False,False,False,False,True,False
4,15.554293,4079.0,85.013188,68.521885,115.785714,33.128355,0.201974,2.432218,0.592206,0.005535,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,7.783232,212.0,70.289424,58.684552,5.357143,26.794008,0.558049,2.629016,0.193424,0.332677,...,False,False,False,False,False,False,False,False,True,False
1047,17.327509,1044.0,210.529395,174.739314,10.071429,34.052881,0.164406,3.094639,0.510198,0.274645,...,False,False,False,False,False,False,False,True,False,False
1048,1.200506,1040.0,235.563711,190.538695,3.571429,38.025742,0.371263,3.073274,0.813218,0.422813,...,False,False,False,False,False,False,False,True,False,False
1049,1.076817,324.0,239.301695,192.178675,6.428571,35.851558,0.519561,2.967437,0.739173,0.320380,...,False,False,False,False,True,False,False,False,False,False


In [22]:
# transformed_X_train.to_csv("NO_Stan_train_data.csv")
# transformed_X_test.to_csv("NO_Stan_test_data.csv")

#### Standardization

In [ ]:
ss=StandardScaler()
ss.fit(transformed_X_train)

OHE_X_train=pd.DataFrame(ss.transform(transformed_X_train),columns=transformed_X_train.columns)
OHE_y_train = y_train

OHE_X_train.shape

In [ ]:
OHE_X_test=pd.DataFrame(ss.transform(transformed_X_test),columns=transformed_X_test.columns)
OHE_X_test.shape

In [25]:
# OHE_X_train.to_csv("OHE_train_data.csv")
# OHE_X_test.to_csv("OHE_test_data.csv")

CSV files of the transformed datasets are cleaned and saved, which can be used for running through the various algorithms.

### Create Log(Sales) response variable

An alternative approach to using Sales as the response variable was explored by using Log(Sales).

#### Plot of distribution of Sales

In [ ]:
plt.figure(figsize=(12,12))
plt.hist(y_train,bins=150,histtype='stepfilled',color='blue')

plt.ylabel("Frequency")
plt.xlabel("Sales")
plt.show()

In [ ]:
plt.boxplot(y_train)
plt.show()

In [ ]:
print('mean of y_train: ',y_train.mean())
print('max of y_train: ',y_train.max())


Observation: Since there are an extreme range of Sales values, log(Sales) as a response variable may be more suitable to approximately conform to near normality.

#### Plot of distribution of Log(Sales)

In [ ]:
plt.figure(figsize=(12,12))
plt.hist(np.log(y_train),bins=150,histtype='stepfilled',color='blue')

plt.ylabel("Frequency")
plt.xlabel("log(Sales)")
plt.show()

In [ ]:
plt.boxplot(np.log(y_train))
plt.show()

In [ ]:
print('mean of y_train: ',np.log(y_train).mean())
print('max of y_train: ',np.log(y_train).max())


#### Creating alternate y_train_log dataframe

Log(Sales) will be used as an alternate response variable.

In [32]:
y_train_log = np.log(y_train)

## Testing various models for Sales prediction

### Random Forests

#### Grid Search

In [33]:
# # Run grid search

# from sklearn.model_selection import GridSearchCV
# import numpy as np

# num_features = OHE_X_train.shape[1]
# print("Number of features:", num_features)


# # Define the parameter grid to search
# param_grid = {
#     'n_estimators': [1000, 1500],  # Number of trees in the forest
#     'ccp_alpha': [0.02, 0.04, 0.06],  # Learning rate
#     'max_depth': [3, 4],  # Maximum depth of the individual trees
#     'min_samples_split': [2, 3],  # Minimum samples required to split an internal node
#     'min_samples_leaf': [1, 2],  # Minimum samples required to be at a leaf node
#     'max_features': ['sqrt', 'log2', int(num_features/3)]
# }

# # Create the RandomBoostingRegressor
# rf_regressor = RandomForestRegressor()

# # Create the GridSearchCV object with cross-validation
# grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

# # Perform the grid search on your data
# # X_train and y_train are your training data
# grid_search.fit(OHE_X_train, OHE_y_train)

# # Print the best hyperparameters and corresponding score
# print("Best Hyperparameters: ", grid_search.best_params_)
# print("Best Score: ", grid_search.best_score_)

In [ ]:
kf = KFold(n_splits=10)
sum = 0
for train_index, test_index in kf.split(OHE_X_train):
    XX_train = OHE_X_train.iloc[train_index]
    YY_train = OHE_y_train.iloc[train_index]

    XX_test = OHE_X_train.iloc[test_index]
    YY_test = OHE_y_train.iloc[test_index]
    rf = RandomForestRegressor(n_estimators=1500, ccp_alpha=0.03, max_features=51, min_samples_leaf=1, min_samples_split=2)
    rf.fit(XX_train,YY_train)
    sum += MSE(rf.predict(XX_test),YY_test)
print("Random Forest Best Parameters MSE= ", sum/10)
print()

#### Best Results

### SKLearn Gradient Boosting Regressor

#### Grid Search

In [35]:
# from sklearn.model_selection import GridSearchCV
# import numpy as np

# # Define the parameter grid to search
# param_grid = {
#     'n_estimators': [25, 50, 75, 100, 150, 200, 250, 300],  # Number of trees in the forest
#     'learning_rate': [0.01, 0.02, 0.1, 0.2, 0.4],  # Learning rate
#     'max_depth': [3, 4, 5, 6, 7, 8, 9],  # Maximum depth of the individual trees
#     'min_samples_split': [10, 15, 20],  # Minimum samples required to split an internal node
#     'min_samples_leaf': [2, 4, 6, 8],  # Minimum samples required to be at a leaf node
#     'max_features': ['sqrt', 'log2']
# }

# # Create the GradientBoostingRegressor
# gb_regressor = GradientBoostingRegressor()

# # Create the GridSearchCV object with cross-validation
# grid_search = GridSearchCV(estimator=gb_regressor, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

# # Perform the grid search on your data
# # X_train and y_train are your training data
# grid_search.fit(OHE_X_train, OHE_y_train)

# # Print the best hyperparameters and corresponding score
# print("Best Hyperparameters: ", grid_search.best_params_)
# print("Best Score: ", grid_search.best_score_)

#### Best results

In [ ]:
from  sklearn.ensemble import GradientBoostingRegressor

kf = KFold(n_splits=10)
sum = 0
for train_index, test_index in kf.split(OHE_X_train):
    XX_train = OHE_X_train.iloc[train_index]
    YY_train = OHE_y_train.iloc[train_index]

    XX_test = OHE_X_train.iloc[test_index]
    YY_test = OHE_y_train.iloc[test_index]
    gbc = GradientBoostingRegressor(learning_rate=0.2, n_estimators=100, max_depth=3, min_samples_leaf=2, min_samples_split=10)
    gbc.fit(XX_train,YY_train)
    sum += MSE(gbc.predict(XX_test),YY_test)
print("MSE= ", sum/10)
print()

### CatBoost Regressor

#### Grid Search

In [37]:
# from catboost import CatBoostRegressor, Pool, metrics, cv
# import numpy as np

# # Define the parameter grid to search
# param_grid = {
#     'n_estimators': [100, 200, 300], # Number of trees in the forest
#     'learning_rate': [0.01, 0.1, 0.2], # Learning rate
#     'max_depth': [3, 4, 5], # Maximum depth of the individual trees
#     'min_child_samples': [2, 5, 10], # Minimum samples required to split an internal node
#     'l2_leaf_reg': [1, 3, 5, 7, 9], # Minimum samples required to be at a leaf node
#     }

# # Create the CatBoostingRegressor
# cb_regressor = CatBoostRegressor()

# # Create the GridSearchCV object with cross-validation
# grid_search_result = cb_regressor.grid_search(param_grid, X=OHE_X_train, y=OHE_y_train, cv=10)
# grid_search_result

#### Best Results

In [ ]:
from catboost import CatBoostRegressor

kf = KFold(n_splits=10)
sum = 0
for train_index, test_index in kf.split(OHE_X_train):
    XX_train = OHE_X_train.iloc[train_index]
    YY_train = OHE_y_train.iloc[train_index]

    XX_test = OHE_X_train.iloc[test_index]
    YY_test = OHE_y_train.iloc[test_index]
    cbr = CatBoostRegressor(min_data_in_leaf = 2, depth = 3, l2_leaf_reg = 3, iterations = 200, learning_rate = 0.2)
    cbr.fit(XX_train,YY_train)

    cbr_predictions = cbr.predict(XX_test)
    cbr_predictions = np.maximum(cbr_predictions, 0)

    sum += MSE(cbr_predictions,YY_test)
print("MSE= ", sum/10)
print()

### XGBoost Regressor

In [39]:
#### Grid Search

In [40]:
# from xgboost import XGBRegressor
# from sklearn.model_selection import GridSearchCV

# param_grid={"learning_rate": (0.05, 0.10, 0.15),
#             "max_depth": [ 3, 4, 5, 6, 8],
#             "min_child_weight": [ 1, 3, 5, 7],
#             "gamma":[ 0.0, 0.1, 0.2],
#             "colsample_bytree":[ 0.3, 0.4],
#             }

# grid_search = GridSearchCV(
#             estimator=XGBRegressor(),
#             param_grid=param_grid,
#             cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

# grid_search.fit(OHE_X_train, OHE_y_train)

# print("Best Hyperparameters: ", grid_search.best_params_)
# print("Best Score: ", grid_search.best_score_)

#### Best Results

In [ ]:
from xgboost import XGBRegressor

kf = KFold(n_splits=10)
sum = 0
for train_index, test_index in kf.split(OHE_X_train):
    XX_train = OHE_X_train.iloc[train_index]
    YY_train = OHE_y_train.iloc[train_index]

    XX_test = OHE_X_train.iloc[test_index]
    YY_test = OHE_y_train.iloc[test_index]
    xgr = XGBRegressor(colsample_bytree=0.4, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=1)
    xgr.fit(XX_train,YY_train)
    sum += MSE(xgr.predict(XX_test),YY_test)
print("MSE= ", sum/10)
print()

### Linear Regression

#### Lasso

In [ ]:
from sklearn.linear_model import Lasso

kf = KFold(n_splits=10)
alphas = [0.01,0.1,1,5,10,50,100]
for alpha in alphas:
    sum = 0
    for train_index, test_index in kf.split(OHE_X_train):
        XX_train = OHE_X_train.iloc[train_index]
        YY_train = OHE_y_train.iloc[train_index]

        XX_test = OHE_X_train.iloc[test_index]
        YY_test = OHE_y_train.iloc[test_index]

        lasso=Lasso(alpha=alpha)
        lasso.fit(XX_train, YY_train)
        sum += MSE(lasso.predict(XX_test),YY_test)
    print("Alpha: ", alpha, " - MSE= ", sum/10)
    print()

##### Best Results

Parameters: Alpha:  5

MSE =  1412.8959793459783

#### Ridge

In [ ]:
from sklearn.linear_model import Ridge

kf = KFold(n_splits=10)
alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
for alpha in alphas:
    sum = 0
    for train_index, test_index in kf.split(OHE_X_train):
        XX_train = OHE_X_train.iloc[train_index]
        YY_train = OHE_y_train.iloc[train_index]

        XX_test = OHE_X_train.iloc[test_index]
        YY_test = OHE_y_train.iloc[test_index]

        ridge=Ridge(alpha=alpha)
        ridge.fit(XX_train, YY_train)
        sum += MSE(ridge.predict(XX_test),YY_test)
    print("Alpha: ", alpha, " - MSE= ", sum/10)
    print()

##### Best Results

Parameters: Alpha:  100

MSE =  1366.20383362699

### SVM (Best model)

#### Grid Search

In [44]:
# svm_model = svm.SVR()

# param_grid = {
#     'C' : [0.1, 1, 10, 100, 1000],
#     'kernel': ['linear', 'rbf'],
#     'gamma': ['scale', 'auto']
# }
# grid_search = GridSearchCV(svm_model, param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs = -1, cv=10, return_train_score=True)
# grid_search.fit(Stan_X_train_df, Stan_y_train_df)
# res = grid_search.cv_results_['mean_test_score'][grid_search.best_index_]

# print (grid_search.best_params_)
# print (res)

As the GridSearch with SVM has high time complexity, the results are as follows:

Best parameter for the linear svm model is {'C': 1000, 'gamma': 'auto', 'kernel': 'rbf'}
 corresponding MSE for the given parameters is -779.1463478155191

In [45]:
# svm_model = svm.SVR()

# # Define the parameter grid for GridSearchCV
# param_grid = {
#     'C' : [0.1, 1, 10, 100, 1000],
#     'kernel': ['poly', 'rbf'],
#     'degree':[2, 3, 4],  # other degree is omitted for now due to time comp
#     'gamma': ['scale', 'auto']
# }

# # k-fold cross-validation
# num_splits = 10
# kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)

# # Initialize GridSearchCV
# grid_search = GridSearchCV(svm_model, param_grid, scoring='neg_mean_squared_error', n_jobs = -1, cv=kf, return_train_score=True)  # 10-fold cross-validation

# # Perform grid search with k-fold cross-validation to find the best hyperparameters
# grid_search.fit(Stan_X_train_df, Stan_y_train_df)
# mse = grid_search.cv_results_['mean_test_score'][grid_search.best_index_]


# # Print the best hyperparameters and corresponding accuracy score
# print (f' Best parameter for the linear svm model is {grid_search.best_params_}')
# print (f' corresponding MSE for the given parameters is {mse}')

As the GridSearch with SVM has high time complexity, the results are as follows:

Best parameter for the linear svm model is {'C': 1000, 'gamma': 'auto', 'kernel': 'rbf'}
 corresponding MSE for the given parameters is -779.1463478155191

#### Best results


In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
total_mse = 0

for train_index, test_index in kf.split(Stan_X_train_df):
    XX_train, XX_test = Stan_X_train_df.iloc[train_index], Stan_X_train_df.iloc[test_index]
    YY_train, YY_test = Stan_y_train_df.iloc[train_index]['sales'], Stan_y_train_df.iloc[test_index]['sales']

    svm_model = SVR(C=1000, gamma='auto', kernel='rbf')
    svm_model.fit(XX_train, YY_train)

    svm_predictions = svm_model.predict(XX_test)
    svm_predictions = np.maximum(svm_predictions, 0)

    mse = MSE(svm_predictions, YY_test)
    total_mse += mse

average_mse = total_mse / 10
print("Average MSE:", average_mse)

In [22]:
kf = KFold(n_splits=10, shuffle=True)
total_mse = 0

for train_index, test_index in kf.split(OHE_X_train):
    XX_train = OHE_X_train.iloc[train_index]
    YY_train = OHE_y_train.iloc[train_index]

    XX_test = OHE_X_train.iloc[test_index]
    YY_test = OHE_y_train.iloc[test_index]

    svm_model = SVR(C=1000, gamma='auto', kernel='rbf')
    svm_model.fit(XX_train, YY_train)

    svm_predictions = svm_model.predict(XX_test)
    svm_predictions = np.maximum(svm_predictions, 0)

    mse = MSE(svm_predictions, YY_test)
    total_mse += mse

average_mse = total_mse / 10
print("Average MSE:", average_mse)

Average MSE: 1369.5926750742092


We attempted to reduce the MSE by only using a subset of the predictors. The result was quite a substantial improvement in MSE. The predictors that provided the best MSE are ['clickVolume', 'avgFinalUnitPrice', 'ma14SalesVolume', 'attribute1', 'attribute2', 'meanUserLevel'].

In [23]:
chosen_column = ['clickVolume', 'avgFinalUnitPrice', 'ma14SalesVolume', 'attribute1', 'attribute2', 'meanUserLevel']
Stan_X_train_df_filtered = Stan_X_train_df[chosen_column]
Stan_X_test_df_filtered = Stan_X_test_df[chosen_column]

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
total_mse = 0

for train_index, test_index in kf.split(Stan_X_train_df):
    XX_train, XX_test = Stan_X_train_df_filtered.iloc[train_index], Stan_X_train_df_filtered.iloc[test_index]
    YY_train, YY_test = Stan_y_train_df.iloc[train_index]['sales'], Stan_y_train_df.iloc[test_index]['sales']

    svm_model = SVR(C=1000, gamma='auto', kernel='rbf')
    svm_model.fit(XX_train, YY_train)

    svm_predictions = svm_model.predict(XX_test)
    svm_predictions = np.maximum(svm_predictions, 0)

    mse = MSE(svm_predictions, YY_test)
    total_mse += mse

average_mse = total_mse / 10
print("Average MSE:", average_mse)

## Sales prediction

For our sales prediction, we are using the SVM model because it provided us with the lowest mean squared error in our testing. The same hyperparameters are used as during our testing and the same subset of predictors that provided the lowest MSE are used. The Stan_X processed dataset is used as it provided better performance for the SVM model.

In [ ]:
Data_test_w_prediction = Data_test.copy()

svm_model = SVR(C=1000, gamma='auto', kernel='rbf')

svm_model.fit(Stan_X_train_df_filtered, Stan_y_train_df)

svm_predictions = svm_model.predict(Stan_X_test_df_filtered)
svm_predictions = np.maximum(svm_predictions, 0)

Data_test_w_prediction['sales'] = np.round(svm_predictions)

Data_test_w_prediction.head()

## Inventory decision

For our inventory decision, we first decided to find out how the error between our prediction and the actual sales for the training set was distributed. Then we could utilize the optimal inventory decision formula for that distribution. We plotted the cumulative distribution function for our error, as well as for an estimate of the best fitting Normal, Exponential and Uniform Distribution curves.  

In [ ]:
# Calculate the error between our predictions and actual sales for the training set
error_data = (Stan_y_train_df['sales']-svm_model.predict(Stan_X_train_df_filtered))

# Sort the data for plotting the empirical CDF
sorted_error_data = np.sort(error_data)
ecdf = np.arange(1, len(sorted_error_data) + 1) / len(sorted_error_data)

# Plot the empirical CDF
plt.step(sorted_error_data, ecdf, label='Empirical Error CDF', color='g', where='post')

# Plot the best-fitting normal distribution CDF
mu, std = norm.fit(error_data)
x = np.linspace(min(error_data), max(error_data), 100)
plt.plot(x, norm.cdf(x, mu, std), 'k', label='Best Fit Normal CDF')

# Plot the best-fitting exponential distribution CDF
lambda_est = 1/np.mean(error_data)
plt.plot(x, expon.cdf(x, scale=1/lambda_est), 'r', label='Best Fit Exponential CDF')

# Plot the best-fitting uniform distribution CDF
min_val, max_val = np.min(error_data), np.max(error_data)
uniform_params = min_val, max_val - min_val
plt.plot(x, uniform.cdf(x, *uniform_params), 'b', label='Best Fit Uniform CDF')

plt.title('Cumulative Distribution Between Prediction and Actual Sales')
plt.xlabel('Error (between prediction and actual sales)')
plt.ylabel('Cumulative Probability')
plt.legend()
plt.show()

From the plot, it was clear that either a Normal Distribution or Exponential Distribution best fit our error distribution. We decided to explore both approaches further.

In [ ]:
# Calculating Overage and Underage Cost

Price=20
Cost=12
Salvage=8
Over=Cost - Salvage
Under=Price - Cost
Over, Under

### Assuming Sales is normally distributed

The first approach is to assume the Sales is normally distributed and estimate the standard deviation based on the samples. For our optimal inventory decision, we used the sales prediction for each row as the mu, and estimated the standard deviation of the sales based on the training dataset.

In [ ]:
# Calculate an estimate of standard deviation based on training dataset

inventory_decision_test = Stan_y_train_df.copy()

ndata = len(Stan_X_train_df_filtered)
nfeatures = len(Stan_X_train_df_filtered.columns)

std_est = np.sqrt(np.sum((svm_model.predict(Stan_X_train_df_filtered)-Stan_y_train_df['sales'])**2)/(ndata-nfeatures-1))

# Apply optimal decision formula (normal distribution assumption) to training dataset, with sales prediction taken as mu

inventory_decision_test['normal_distribution_inventory'] = np.ceil(sct.norm.ppf(Under/(Under+Over))*std_est + svm_model.predict(Stan_X_train_df_filtered))
inventory_decision_test.head()

### Assuming Sales is exponentially distributed

The second approach is to assume the Sales is exponentially distributed and estimate the lambda based on the samples. Our estimate of the lambda is 1/mean(residual_error) from the training dataset. Using this we calculate the y* and add it to our sales prediction. We also subtract the mean of the residual error from this amount, giving our inventory prediction.

In [ ]:
# Calculate an estimate of lambda based on training dataset

error_data = (Stan_y_train_df['sales']-svm_model.predict(Stan_X_train_df_filtered))
lambda_est = 1/np.mean(error_data)


# Apply optimal decision formula (exponential distribution assumption) to training dataset

y_star = (-1/lambda_est)*np.log(1-(Under/(Under+Over)))

inventory_decision_test['exponential_distribution_inventory'] = np.ceil(y_star - np.mean(error_data) + svm_model.predict(Stan_X_train_df_filtered))
inventory_decision_test.head()


### Evaluating profit from both approaches

The profit achieved from using both approaches on the training dataset is calculated. The actual sales tell us the demand.

In [ ]:
# Test profit on training dataset
def profit(sales,inv):
    return (Price-Salvage)*np.minimum(sales,inv)-(Cost-Salvage)*inv

inventory_decision_test['normal_distribution_profit'] = profit(inventory_decision_test['sales'], inventory_decision_test['normal_distribution_inventory'])
inventory_decision_test['exponential_distribution_profit'] = profit(inventory_decision_test['sales'], inventory_decision_test['exponential_distribution_inventory'])

print(f"Profit from Normal Distribution approach: {np.sum(inventory_decision_test['normal_distribution_profit'])}")
print(f"Profit from Exponential Distribution approach: {np.sum(inventory_decision_test['exponential_distribution_profit'])}")

Seeing as how the exponential distribution approach managed to give us a greater overall profit, we decided to use this approach for the test dataset inventory prediction.

### Exponential Distribution approach for test datset

In [37]:
# Calculate an estimate of lambda based on training dataset

error_data = (Stan_y_train_df['sales']-svm_model.predict(Stan_X_train_df_filtered))
lambda_est = 1/np.mean(error_data)

# Apply optimal decision formula (exponential distribution assumption) to training dataset

y_star = (-1/lambda_est)*np.log(1-(Under/(Under+Over)))

Data_test_w_prediction['inventory'] = np.ceil(y_star - np.mean(error_data) + Data_test_w_prediction['sales'])
Data_test_w_prediction.head()

,productID,brandID,attribute1,attribute2,attribute3,attribute4,clickVolume,avgOriginalUnitPrice,avgFinalUnitPrice,ma14SalesVolume,...,meanAge,gender,meanEducation,maritalStatus,plus,meanPurchasePower,meanUserLevel,meanCityLevel,sales,inventory
0,77,38,3.0,100.0,2,5.670374,173.0,180.835683,165.459988,12.857143,...,26.960521,0.053158,2.958528,0.668194,0.148227,2.347953,2.638135,1.997034,5.0,6.0
1,47,30,4.0,100.0,9,5.191429,142.0,190.802856,138.995392,2.285714,...,33.111475,0.317086,3.292282,0.709143,0.272464,2.409795,3.973464,1.385002,1.0,2.0
2,33,25,3.0,100.0,1,2.461109,144.0,127.385574,123.286542,4.500000,...,30.353550,0.000000,2.668048,1.002511,0.000000,1.955541,2.937542,2.369596,3.0,4.0
3,68,28,3.0,100.0,2,4.659192,220.0,240.873276,168.258175,17.500000,...,29.692379,0.000000,2.470400,0.424764,0.000000,1.932938,2.480643,2.505191,6.0,7.0
4,65,16,3.0,90.0,2,15.554293,4079.0,85.013188,68.521885,115.785714,...,33.128355,0.201974,2.432218,0.592206,0.005535,2.428117,1.792081,2.669685,100.0,101.0


## Create output csv file of predictions

In [38]:
# Create output csv file with name group09.csv

Data_test_w_prediction.to_csv('group09.csv', index=True)